In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import EfficientNetB3

In [2]:
base_dir = '../data/telemoji'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_datagen = ImageDataGenerator(
#     rescale=1/255,    
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

validation_datagen = ImageDataGenerator(
#     rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224,224),
    batch_size=100,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size=(224,224),
    batch_size=100,
    class_mode='categorical'
)

# pretrained network
model_base = EfficientNetB3(include_top=False, 
                            weights='imagenet', input_shape=(224,224,3))
# model_base의 weight학습을 동결
model_base.trainable=False

model_base.summary()

Found 16800 images belonging to 7 classes.
Found 7000 images belonging to 7 classes.
Model: "efficientnetb3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              


In [4]:
model = Sequential()

# pretrained network를 우리의 모델 앞에 추가
model.add(model_base)

model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=7, activation='softmax'))

model.summary()

model.compile(optimizer=RMSprop(learning_rate=1e-5), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb3 (Functional)  (None, 7, 7, 1536)        10783535  
_________________________________________________________________
flatten_1 (Flatten)          (None, 75264)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               38535680  
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 7)                

In [5]:
history = model.fit(train_generator, steps_per_epoch=24, epochs=50,
                    validation_data=validation_generator,
                    validation_steps=10)

Epoch 1/50
24/24 [==============================] - 41s 2s/step - loss: 2.2381 - accuracy: 0.1521 - val_loss: 1.9248 - val_accuracy: 0.2100
Epoch 2/50
24/24 [==============================] - 39s 2s/step - loss: 2.1045 - accuracy: 0.1708 - val_loss: 1.9230 - val_accuracy: 0.2020
Epoch 3/50
24/24 [==============================] - 39s 2s/step - loss: 2.0408 - accuracy: 0.1933 - val_loss: 1.8789 - val_accuracy: 0.2510
Epoch 4/50
24/24 [==============================] - 40s 2s/step - loss: 1.9901 - accuracy: 0.2079 - val_loss: 1.8645 - val_accuracy: 0.2520
Epoch 5/50
24/24 [==============================] - 41s 2s/step - loss: 1.9666 - accuracy: 0.2108 - val_loss: 1.8640 - val_accuracy: 0.2590
Epoch 6/50
24/24 [==============================] - 41s 2s/step - loss: 1.9711 - accuracy: 0.2096 - val_loss: 1.8550 - val_accuracy: 0.2820
Epoch 7/50
24/24 [==============================] - 40s 2s/step - loss: 1.9388 - accuracy: 0.2142 - val_loss: 1.8528 - val_accuracy: 0.2540
Epoch 8/50
24/24 [==